In [ ]:
import os
import pyvtk
import numpy as np
from netCDF4 import Dataset

In [ ]:
# data dir
datadir = 'output/stations/inplane_stations'

# read rank-station info
rank_station_info = np.loadtxt('%s/rank_station.info' % datadir, dtype=str, skiprows=1)
rank_station_dict = {}
station_key_dict = {}

nstation = len(rank_station_info)
for index, item in enumerate(rank_station_info):
    rank = int(item[0])
    station_key = item[1]
    if rank not in rank_station_dict.keys():
        rank_station_dict[rank] = []
    rank_station_dict[rank].append(station_key)
    station_key_dict[station_key] = index

In [ ]:
# open
nc = []
for rank in rank_station_dict.keys():
    nc.append(Dataset('%s/axisem3d_synthetics.nc.rank%d' % (datadir, rank), 'r'))

# time
time = nc[0].variables['time_points'][:]
ntime = len(time)

In [ ]:
# data
data = np.ndarray((ntime, nstation))

pos = 0
for irank, rank in enumerate(rank_station_dict.keys()):
    nstation_on_rank = len(rank_station_dict[rank])
    data[:, pos:(pos+nstation_on_rank)] = nc[rank].variables['data'][:, 0, :]
    pos += nstation_on_rank

In [ ]:
###### vtk grid ######

# surface z
surface_z = 6370e3

# x of stations
x_all = np.arange(0, 5000, 50)

# depth of station
depth_all = np.arange(0, 3801, 50)

# grid xyz and connectivity
xyz = np.ndarray((nstation, 3))
connect = []

for ix, x in enumerate(x_all):
    for idep, dep in enumerate(depth_all):
        st_key = 'INPLANE.st_x%d_d%d' % (x, dep)
        st_index = station_key_dict[st_key]
        xyz[st_index, :] = [x, 0, surface_z - dep]
        if ix != len(x_all) - 1 and idep != len(depth_all) - 1:
            connect.append([st_index, 
                            station_key_dict['INPLANE.st_x%d_d%d' % (x + 50, dep)], 
                            station_key_dict['INPLANE.st_x%d_d%d' % (x + 50, dep + 50)], 
                            station_key_dict['INPLANE.st_x%d_d%d' % (x, dep + 50)]])
connect = np.array(connect)

# points
vtk_points = pyvtk.UnstructuredGrid(list(zip(xyz[:,0], xyz[:,1], xyz[:,2])), quad=connect)
    
############ vtk ############
os.makedirs('%s/vtk' % datadir, exist_ok=True)

for istep in np.arange(ntime):
    # vtk
    vtk = pyvtk.VtkData(vtk_points, 
                        pyvtk.PointData(pyvtk.Scalars(data[istep,:], name='U_NORM')), 
                        'disp_norm')
    vtk.tofile('%s/vtk/U_NORM.%d.vtk' % (datadir, istep), 'binary')

    